In [ ]:
import pandas as pd
from tqdm import tqdm

# Function to generate file paths for a given prefix (hc_1 or hc_5)
def generate_file_paths(prefix):
    base_files = [
        '2000.csv', '2100.csv', '2200.csv',
        '2300.csv', '2400.csv', '2500.csv',
        '2600.csv', '2700.csv', '2800.csv',
        '2900.csv', '3000.csv'
    ]
    return [f'data/{prefix}_{f}' for f in base_files]

# Generate file paths for both hc_1 and hc_5
file_paths_hc1 = generate_file_paths('hc_1')
output_file_paths_hc1 = [path.replace('hc_1', 'filtered_hc_1') for path in file_paths_hc1]

file_paths_hc5 = generate_file_paths('hc_5')
output_file_paths_hc5 = [path.replace('hc_5', 'filtered_hc_5') for path in file_paths_hc5]

file_paths_hc10 = generate_file_paths('hc_10')
output_file_paths_hc10 = [path.replace('hc_10', 'filtered_hc_10') for path in file_paths_hc10]


# Combine both sets of file paths
combined_file_paths = file_paths_hc1 + file_paths_hc5 + file_paths_hc10
combined_output_paths = output_file_paths_hc1 + output_file_paths_hc5 + output_file_paths_hc10

# Iterate over the combined files with a progress bar
for i in tqdm(range(len(combined_file_paths)), desc="Processing files"):
    # Read the CSV file
    df = pd.read_csv(combined_file_paths[i])

    # Filter the dataframe for specific PID numbers
    filtered_df = df[df['PID'].isin([5, -5, 11, -11, 13, -13])]

    # Save the filtered dataframe to a new CSV file
    filtered_df.to_csv(combined_output_paths[i], index=False)

    print(f"Filtered data saved to {combined_output_paths[i]}")


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# Function to generate file paths for a given prefix (filtered_hc_1, filtered_hc_5, or filtered_hc_10)
def generate_file_paths(prefix):
    base_files = [
        '2000.csv', '2100.csv', '2200.csv',
        '2300.csv', '2400.csv', '2500.csv',
        '2600.csv', '2700.csv', '2800.csv',
        '2900.csv', '3000.csv'
    ]
    return [f'data/{prefix}_{f}' for f in base_files]

# Generate file paths for the filtered files
filtered_file_paths_hc1 = generate_file_paths('filtered_hc_1')
filtered_file_paths_hc5 = generate_file_paths('filtered_hc_5')
filtered_file_paths_hc10 = generate_file_paths('filtered_hc_10')

# Combine all file paths
combined_filtered_paths = filtered_file_paths_hc1 + filtered_file_paths_hc5 + filtered_file_paths_hc10

# Number of PID groups
pid_groups = ['PID 11 and 13', 'PID -11 and -13', 'PID 5', 'PID -5']

# Define the bin ranges for Eta and Phi
eta_range = (-10, 10)
phi_range = (-np.pi, np.pi)
bins = 50

# Process each file separately
for i, file_path in tqdm(enumerate(combined_filtered_paths), total=len(combined_filtered_paths), desc="Creating Eta-Phi heatmaps"):
    # Read the filtered CSV file
    df = pd.read_csv(file_path)

    # Grouping PID values
    groups = {
        'PID 11 and 13': df[df['PID'].isin([11, 13])],
        'PID -11 and -13': df[df['PID'].isin([-11, -13])],
        'PID 5': df[df['PID'] == 5],
        'PID -5': df[df['PID'] == -5]
    }

    # Create a figure with subplots for each PID group
    fig, axes = plt.subplots(1, len(pid_groups), figsize=(20, 5))
    fig.suptitle(f'Eta-Phi Heatmaps ({file_path.split("/")[-1].replace(".csv", "")})')

    # Plot Eta vs Phi heatmap for each group
    for j, (label, group) in enumerate(groups.items()):
        ax = axes[j]
        heatmap = ax.hist2d(group['Eta'], group['Phi'], bins=[bins, bins], range=[eta_range, phi_range], cmap='viridis')
        ax.set_xlabel('Eta')
        ax.set_ylabel('Phi')
        ax.set_title(label)
        ax.grid(True)

        # Add colorbar
        fig.colorbar(heatmap[3], ax=ax, label='Density')

    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.96])

    # Save the plot as an image for each file
    plt.savefig(f'Heatmaps_{file_path.split("/")[-1].replace(".csv", "")}.pdf')
    plt.close(fig)  # Close the figure to free up memory

# Combine individual plots into a final combined plot if needed
# This step would require loading saved images or saving all plots to the same figure if needed.


Creating Eta-Phi heatmaps:   0%|          | 0/33 [00:00<?, ?it/s]

Creating Eta-Phi heatmaps: 100%|██████████| 33/33 [01:57<00:00,  3.55s/it]
